In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np

In [2]:
def add_binary (high_low_list,length):
     if high_low_list[length-1] == 0: 
          high_low_list[length-1] = 1
          return high_low_list
     else: 
          high_low_list[length-1] = 0
          return add_binary(high_low_list,length-1)

In [3]:
def make_output (high_low_list, df_out):
     df_out.append(list(high_low_list))
     number_of_ones = 0
     for i in range(len(high_low_list)):
          if high_low_list[i] == 1: number_of_ones += 1
     if number_of_ones == len(high_low_list): return df_out

     length = len(high_low_list)
     high_low_list = add_binary(high_low_list,length)

     return make_output(high_low_list, df_out)

In [4]:
files = 1000 # number of solution files to process
# dataset = pd.read_csv('dataset_multinode.csv')
# dataset = np.array()
freq_total_array =[]
number_of_nodes = []

cfg_name = 'TS{}.cfg'
cfg_list = []
col_list_full = []
for i in range(0,files):
     cfg_list.append(open(cfg_name.format(i)))
     lines = cfg_list[i].readlines()
     number_of_nodes.append(int(lines[30][14]+lines[30][15]))
     col_list_single = []
     for j in range(number_of_nodes[i]):
          if j == 9: col_list_single.append(lines[31+j][3])
          else: col_list_single.append(lines[31+j][2])
     col_list_full.append(col_list_single)

topo_name = 'TS{}.topo'
topo_list = []
for i in range(0,files):
     # if i == 1 or i == 10 or i == 13 or i == 23 or i == 26: continue
     topo_list.append(pd.read_csv(topo_name.format(i), sep = '\t'))
     topo_list[i]

df_name = 'TS{}_solution.dat'
df_list = []
for i in range(0,files):
     
     # if i == 1 or i == 10 or i == 13 or i == 23 or i == 26: continue 
     if number_of_nodes[i] == 4: df_list.append(pd.read_csv(df_name.format(i), sep="\t", names=["serial","states","1","2","3","4"]))
     if number_of_nodes[i] == 5: df_list.append(pd.read_csv(df_name.format(i), sep="\t", names=["serial","states","1","2","3","4","5"]))
     if number_of_nodes[i] == 6: df_list.append(pd.read_csv(df_name.format(i), sep="\t", names=["serial","states","1","2","3","4","5","6"]))
     if number_of_nodes[i] == 7: df_list.append(pd.read_csv(df_name.format(i), sep="\t", names=["serial","states","1","2","3","4","5","6","7"]))
     if number_of_nodes[i] == 8: df_list.append(pd.read_csv(df_name.format(i), sep="\t", names=["serial","states","1","2","3","4","5","6","7","8"]))
     if number_of_nodes[i] == 9: df_list.append(pd.read_csv(df_name.format(i), sep="\t", names=["serial","states","1","2","3","4","5","6","7","8","9"]))
     if number_of_nodes[i] == 10: df_list.append(pd.read_csv(df_name.format(i), sep="\t", names=["serial","states","1","2","3","4","5","6","7","8","9","10"]))

par_name = 'TS{}_parameters.dat'
par_list = []
for i in range(0,files):
     # if i == 1 or i == 10 or i == 13 or i == 23 or i == 26: continue
     par_list.append(pd.read_csv(par_name.format(i), sep="\t", header=None))

for j in range(len(topo_list)):
     
     df1 = df_list[j]

     col_list = col_list_full[j]
     topo = topo_list[j]
     df1 = df_list[j]
     par = par_list[j]
     par_selective_columns = []
     for i in range(2*number_of_nodes[j]+1):
          par_selective_columns.append(i+1)

     par = par[par_selective_columns]
     par = par.rename(columns = {1: 'states'})
     for column in range(number_of_nodes[j]):
          par = par.rename(columns = {column+2: col_list[column]+'p'})
     for column in range(number_of_nodes[j]):
          par = par.rename(columns = {number_of_nodes[j]+column+2: col_list[column]+'d'})
     temp_states = df1['states']

     nodes = [['A',0,0],['B',1,0],['C',2,0],['D',3,0],['E',4,0],['F',5,0],['G',6,0],['H',7,0],['I',8,0],['J',9,0]]
     nodes_orig = nodes
     nodes = nodes[:number_of_nodes[j]]
     node_to_index_dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9}

     for i in range (len(topo)):
          nodes[node_to_index_dict[topo['Source'][i]]][2] += 1
          # nodes[node_to_index_dict[topo['Target'][i]]][2] += 1
          nodes[node_to_index_dict[topo['Source'][i]]].append(node_to_index_dict[topo['Target'][i]])
          nodes[node_to_index_dict[topo['Source'][i]]].append(topo['Type'][i])
          # nodes[node_to_index_dict[topo['Target'][i]]].append(node_to_index_dict[topo['Source'][i]])

     # production/degradation normalization and z normalization of the sultion file
     df2 = np.array(df1.drop(['serial','states'], axis = 'columns'))
     df2 = 2**df2

     dfp = []
     ddf = []
     for i in range(len(df1)):
          temp1 = []
          for col in col_list:
               temp1.append(par[col+'p'][df1.serial[i]-1])
          dfp.append(temp1)

          temp2 = []
          for col in col_list:
               temp2.append(par[col+'d'][df1.serial[i]-1])
          ddf.append(temp2)

     dfp = np.array(dfp)
     ddf = np.array(ddf)
     dfpbyd = dfp/ddf

     df2 = df2/dfpbyd
     df2 = np.log2(df2)
     df3 = np.sum(df2, axis = 0)
     df3 = df3/(len(df2))
     df2 = df2-df3
     
     df1 = pd.DataFrame(df2, columns=col_list)

     df_out = []
     node_names = []
     for i in range(len(nodes)):
          node_names.append(nodes[i][0])
     # df_out.append(node_names)
     freq_total = [0]*((2**len(nodes)))
     high_low_list = []
     for i in range(len(nodes)):
          high_low_list.append(0)
     df_out = make_output(high_low_list,df_out)
     
     df_out = pd.DataFrame(df_out, columns = df_out[0])

     index = len(nodes)
     while index<10:
          df_out[nodes_orig[index][0]] = 0
          index+=1

     df_out = df_out.values.tolist()

     matrix = []
     for i in range(10):
          row = [0]*10
          matrix.append(row)
     
     for i in range(len(nodes)):
          for conn in range(int(len(nodes[i][3:])/2)):
               conn = conn*2
               matrix[i][nodes[i][conn+3]] = nodes[i][conn+4]
     matrix = np.array(matrix)

     matrix_of_matrices = []
     for i in range(len(df_out)):
          temp1 = matrix.copy()
          temp2 = []
          for k in range(10):
               arr = [df_out[i][k]]*10
               temp2.append(arr)
          temp3 = []
          for k in range(10):
               arr = df_out[i]
               temp3.append(arr)
          temp = []
          temp.append(temp1)
          temp.append(temp2)
          temp.append(temp3)
          matrix_of_matrices.append(temp)

     # matrix_of_matrices = []
     # for i in range(len(df_out)-1):
     #      temp = matrix.copy()
     #      array = df_out[i+1]
     #      for k in range(10-len(nodes)):
     #           array.append(0)
     #      np.fill_diagonal(temp,array)
     #      matrix_of_matrices.append(temp)
     # matrix_of_matrices = np.array(matrix_of_matrices)

     # matrix_of_matrices = []
     # for i in range(len(df_out)-1):
     #      temp = matrix.copy()
     #      array = df_out[i+1]
     #      for k in range(10-len(nodes)):
     #           array.append(0)
     #      temp.append(array)
     #      matrix_of_matrices.append(temp)
     # matrix_of_matrices = np.array(matrix_of_matrices)

     for row in range(len(df_out)):
          if row == 0: continue
          for state in range(len(df1)): 
               # for node in range(len(node_names)):
               node = 0
               while node < len(node_names):
                    if df_out[row][node] == 1:
                         if df1[node_names[node]][state]>0: pass
                         else: 
                              node = len(node_names)
                              continue
                    if df_out[row][node] == 0:
                         if df1[node_names[node]][state]<0: pass
                         else:
                              node = len(node_names)
                              continue
                    if node == len(node_names)-1: freq_total[row] += 1
                    node+=1
                    
     # df_out = pd.DataFrame(df_out, columns = df_out[0])
     freq_total = np.array(freq_total)
     freq_total = freq_total/len(df1)
     freq_total = freq_total*10000
     freq_total = freq_total.astype('int')
     # freq_total = np.delete(freq_total,0)

     freq_total_array = np.concatenate([freq_total_array,freq_total])

     if j == 0: dataset = matrix_of_matrices
     else: dataset = np.concatenate([dataset,matrix_of_matrices])

dataset = dataset.reshape(dataset.shape[0],-1)
np.savetxt('dataset_matrix_3d_1000.txt', dataset, fmt = '%d')
np.savetxt('dataset_matrix_3d_1000_out.txt', freq_total_array)

In [14]:
freq_total[16]

2011

In [15]:
matrix_of_matrices[16]

[array([[0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0

In [58]:
len(freq_total)

128

In [53]:
len(matrix_of_matrices)

128

In [32]:
df_out.values.tolist()

[[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 1, 0, 1, 1],
 [0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 0, 1],
 [0, 0, 0, 1, 1, 1, 0],
 [0, 0, 0, 1, 1, 1, 1],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 1, 1],
 [0, 0, 1, 0, 1, 0, 0],
 [0, 0, 1, 0, 1, 0, 1],
 [0, 0, 1, 0, 1, 1, 0],
 [0, 0, 1, 0, 1, 1, 1],
 [0, 0, 1, 1, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 1],
 [0, 0, 1, 1, 0, 1, 0],
 [0, 0, 1, 1, 0, 1, 1],
 [0, 0, 1, 1, 1, 0, 0],
 [0, 0, 1, 1, 1, 0, 1],
 [0, 0, 1, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 1],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 1, 1],
 [0, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 1, 0, 1],
 [0, 1, 0, 0, 1, 1, 0],
 [0, 1, 0, 0, 1, 1, 1],
 [0, 1, 0, 1, 0, 0, 0],
 [0, 1, 0, 1, 0,

In [24]:
x

<function ndarray.tolist>

In [35]:
df_out

,0,0,0,0,0,0,0,H,I,J
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,1,0,0,0
4,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
123,1,1,1,1,0,1,1,0,0,0
124,1,1,1,1,1,0,0,0,0,0
125,1,1,1,1,1,0,1,0,0,0
126,1,1,1,1,1,1,0,0,0,0


In [67]:
dataset[101]

array([[ 0, -1,  0,  0,  0, -1,  0,  0,  0,  0],
       [-1,  0,  0,  0, -1,  0, -1,  0,  0,  0],
       [ 0,  0,  0, -1,  0, -1,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0, -1,  0,  0,  0,  0],
       [-1,  0, -1,  0, -1,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  1,  0,  1,  0,  0,  0]])

In [5]:
matrix_of_matrices

[array([[0, 1, 0, 0, 2, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 2, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 2, 0, 0, 0],
        [0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 2, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 array([[0, 1, 0, 0, 2, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 2, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 2, 0, 0, 0],
        [0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 2, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 2, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 array([[0, 1, 0, 0, 2, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 2, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 2, 0, 0, 0],
        [0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 2, 0, 0, 

In [96]:
len(freq_total_array)

15904

In [92]:
temp

[[[1, -1, 0, 0, -1, 0, 0, 0, 0, 0],
  [-1, 1, 0, -1, -1, 0, 0, 0, 0, 0],
  [0, 0, 1, -1, -1, 0, 0, 0, 0, 0],
  [0, -1, -1, 1, 0, 0, 0, 0, 0, 0],
  [-1, -1, -1, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[1, -1, 0, 0, -1, 0, 0, 0, 0, 0],
  [-1, 1, 0, -1, -1, 0, 0, 0, 0, 0],
  [0, 0, 1, -1, -1, 0, 0, 0, 0, 0],
  [0, -1, -1, 1, 0, 0, 0, 0, 0, 0],
  [-1, -1, -1, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[1, -1, 0, 0, -1, 0, 0, 0, 0, 0],
  [-1, 1, 0, -1, -1, 0, 0, 0, 0, 0],
  [0, 0, 1, -1, -1, 0, 0, 0, 0, 0],
  [0, -1, -1, 1, 0, 0, 0, 0, 0, 0],
  [-1, -1, -1, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0